In [1]:
import os

import rlcard
import torch
from rlcard.agents import NFSPAgentPytorch as NFSPAgent
from rlcard.agents import RandomAgent
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger




Using TensorFlow backend.











In [2]:

# Make environment
env = rlcard.make('no-limit-holdem', config={'seed': 0})
eval_env = rlcard.make('no-limit-holdem', config={'seed': 0})

env.player_num = 2
eval_env.player_num = env.player_num

# Set the iterations numbers and how frequently we evaluate the performance
evaluate_every = 10000
evaluate_num = 1000
episode_num = 1000000

# The intial memory size
memory_init_size = 1000

# Train the agent every X steps
train_every = 64

# The paths for saving the logs and learning curves
log_dir = './experiments/nolimit_holdem_nfsp_result/'
save_dir = 'models/nolimit_holdem_nfsp'

In [3]:

# Set up the agents
agents = []
for i in range(env.player_num):
    agent = NFSPAgent(scope='nfsp' + str(i),
                      action_num=env.action_num,
                      state_shape=env.state_shape,
                      hidden_layers_sizes=[128,128],
                      min_buffer_size_to_learn=memory_init_size,
                      q_replay_memory_init_size=memory_init_size,
                      train_every=train_every,
                      q_train_every = train_every,
                      q_mlp_layers=[128,128],
                    device=torch.device('cpu'))
    agents.append(agent)

try:
    checkpoint = torch.load('models\\initial_model\\model.pth')
    for agent in agents:
        agent.load(checkpoint)
    random_agent = NFSPAgent(scope='nfsp' + str(i),
                      action_num=env.action_num,
                      state_shape=env.state_shape,
                      hidden_layers_sizes=[128,128],
                      min_buffer_size_to_learn=memory_init_size,
                      q_replay_memory_init_size=memory_init_size,
                      train_every=train_every,
                      q_train_every = train_every,
                      q_mlp_layers=[128,128],
                    device=torch.device('cpu'))
    random_agent.load(checkpoint)
    
except FileNotFoundError:
    print('Initial model not found. ')
    random_agent = RandomAgent(action_num=eval_env.action_num)


env.set_agents(agents)
eval_env.set_agents([agents[0]]+ [random_agent for i in range(env.player_num -1)])
logger = Logger(log_dir)


In [4]:

for episode in range(1):#episode_num):

        # First sample a policy for the episode
    for agent in agents:
        agent.sample_episode_policy()

        # Generate data from the environment
    trajectories, _ = env.run(is_training=True)

     # Feed transitions into agent memory, and train the agent
    for i in range(env.player_num):
        for ts in trajectories[i]:
            agents[i].feed(ts)

        # Evaluate the performance. Play with random agents.
    if episode % evaluate_every == 0:
        logger.log_performance(env.timestep, tournament(eval_env, evaluate_num)[0])

    # Close files in the logger
logger.close_files()

    # Plot the learning curve
logger.plot('NFSP')

    # Save model
save_dir = 'models/nolimit_holdem_nfsp_pytorch'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
state_dict = {}
for agent in agents:
    state_dict.update(agent.get_state_dict())



----------------------------------------
  timestep     |  4
  reward       |  2.9514
----------------------------------------
INFO - Agent nfsp0_dqn, step 1000, rl-loss: 4075.569580078125
INFO - Copied model parameters to target network.
INFO - Agent nfsp1_dqn, step 1000, rl-loss: 3952.1650390625
INFO - Copied model parameters to target network.
INFO - Agent nfsp0_dqn, step 15656, rl-loss: 3312.071044921875
----------------------------------------
  timestep     |  31502
  reward       |  5.3046
----------------------------------------
INFO - Agent nfsp1_dqn, step 29928, rl-loss: 3093.855224609375
----------------------------------------
  timestep     |  59015
  reward       |  3.4628
----------------------------------------
INFO - Agent nfsp1_dqn, step 43752, rl-loss: 2472.64501953125
----------------------------------------
  timestep     |  85983
  reward       |  5.3793
----------------------------------------
INFO - Agent nfsp1_dqn, step 58408, rl-loss: 3250.90478515625
-------

KeyboardInterrupt: 

In [5]:
# Save model
save_dir = 'models/nolimit_holdem_nfsp_pytorch'
if not os.path.exists(save_dir): os.makedirs(save_dir)
state_dict = {}
for agent in agents:
    state_dict.update(agent.get_state_dict())
torch.save(state_dict, os.path.join(save_dir, 'model.pth'))